# Compare Auto Segmentation with Crawled

In [ ]:
import os
import json
import numpy as np
import mir_eval
from glob import glob
from fractions import Fraction

import sys
sys.path.append("..")

from utils.common import load_event
from utils.event import expand_score
from utils.align import bar_beat_to_t, get_t_bar
from struct_segment import get_t_fin, get_signature_shift

def get_true_bound_bar(struct):
    bound = []
    for sect in struct['struct']:
        bound += [[entry['start'], entry['end']] for entry in struct['struct'][sect]]
    
    for i, entry in enumerate(bound[:-1]):
        if entry[-1] == bound[i+1][0]:
            bound[i][-1] += .5
            bound[i+1][0] += .5
        else:
            bound[i][-1] += 1

    true_bound = [i[0] for i in bound] + [bound[-1][-1] + 1]
    true_bound = np.array(true_bound) - 1
    return true_bound


def get_sig_shift(composer, base_name):
    event_file = f"../../sonata-dataset-phrase/event/{composer}/{base_name}"
    score_event, pattern = load_event(event_file)
    event, idx_mapping = expand_score(score_event, pattern, repeat_mode="no_repeat")

    map_file = f"../../sonata-dataset-phrase/rendered_midi_no_repeat/{composer}/{base_name}"
    with open(map_file) as f:
        ts_shift = json.load(f)['onset']

    # Renew end time
    t_fin = get_t_fin(max(idx_mapping), ts_shift)
    ts_shift.append({"measure": max(idx_mapping) + 1, "t": float(t_fin),
                     "tempo": ts_shift[-1]['tempo'],
                     "time_signature": ts_shift[-1]['time_signature']})

    # Split sections by tempo, time/key signature shifts, for refining phrase boundaries
    sig_shift = get_signature_shift(event, ts_shift)
    return sig_shift

def get_true_t_bound(struct, sig_shift):
    true_bound = get_true_bound_bar(struct)
    
    t_bound = []
    p = 0
    for bar in true_bound:
    
        if int(bar) == bar:
            bar_beat = (int(bar), 0)
        else:
            while bar > sig_shift[p+1]['measure']:
                p += 1
            beat = Fraction(sig_shift[p]['time_signature']) * 4 * Fraction(1, 2)
            bar_beat = (int(bar), beat)
    
        t_bound.append(float(bar_beat_to_t(bar_beat, sig_shift)))
    t_bound_pair = np.array([t_bound[:-1], t_bound[1:]]).T
    return t_bound_pair

def get_t_bound(pred_struct, sig_shift):
    pred_bound = []
    for sect in pred_struct:
        pred_bound += [entry[0] for entry in pred_struct[sect]]
        pred_bound += [pred_struct[sect][-1][-1]]
    
    pred_t_bound = []
    for entry in pred_bound:
        t = float(bar_beat_to_t((entry[0], Fraction(entry[1])), sig_shift))
        if not len(pred_t_bound) or t > pred_t_bound[-1]:
            pred_t_bound.append(t)
    
    pred_t_bound_pair = np.array([pred_t_bound[:-1], pred_t_bound[1:]]).T
    return pred_t_bound_pair

def get_high_level_t_bound(pred_struct, sig_shift):
    pred_bound = []
    for sect in pred_struct:
        pred_bound.append(pred_struct[sect][0][0])
    pred_bound += [pred_struct[sect][-1][-1]]

    pred_t_bound = []
    for entry in pred_bound:
        t = float(bar_beat_to_t((entry[0], Fraction(entry[1])), sig_shift))
        if not len(pred_t_bound) or t > pred_t_bound[-1]:
            pred_t_bound.append(t)
    
    pred_t_bound_pair = np.array([pred_t_bound[:-1], pred_t_bound[1:]]).T
    return pred_t_bound_pair

def get_high_level_true_bound(struct, sig_shift):
    true_bound = []
    for sect in struct['struct']:
        true_bound.append(struct['struct'][sect][0]['start'] - 1)
    true_bound += [struct['struct'][sect][-1]['end'] - 1]

    t_bound = []
    p = 0
    for bar in true_bound:
    
        if int(bar) == bar:
            bar_beat = (int(bar), 0)
        else:
            while bar > sig_shift[p+1]['measure']:
                p += 1
            beat = Fraction(sig_shift[p]['time_signature']) * 4 * Fraction(1, 2)
            bar_beat = (int(bar), beat)
    
        t_bound.append(float(bar_beat_to_t(bar_beat, sig_shift)))
    t_bound_pair = np.array([t_bound[:-1], t_bound[1:]]).T
    return t_bound_pair

In [ ]:
hr_5f_lst, hrf_lst = [], []
for composer in ['mozart', 'beethoven']:
    for struct_file in sorted(glob(f"./manually_cleaned/{composer}/*.json")):
        
        with open(struct_file) as f:
            struct = json.load(f)

        base_name = os.path.basename(struct_file)
        with open(f"../../sonata-dataset-phrase/struct_mark/{composer}-{base_name}") as f:
            pred_struct = json.load(f)

        sig_shift = get_sig_shift(composer, base_name)
        # t_bound = get_true_t_bound(struct, sig_shift)
        # pred_t_bound = get_t_bound(pred_struct, sig_shift)

        # p_5, r_5, hr_5f = mir_eval.segment.detection(pred_t_bound, t_bound, window=.5)
        # p, r, hrf = mir_eval.segment.detection(pred_t_bound, t_bound, window=3)

        pred_high_level_t_bound = get_high_level_t_bound(pred_struct, sig_shift)
        true_high_level_t_bound = get_high_level_true_bound(struct, sig_shift)

        p_5, r_5, hr_5f = mir_eval.segment.detection(pred_high_level_t_bound, true_high_level_t_bound, window=.5)
        p, r, hrf = mir_eval.segment.detection(pred_high_level_t_bound, true_high_level_t_bound, window=3)
        
        hr_5f_lst.append(hr_5f)
        hrf_lst.append(hrf)

In [ ]:
# High level bounds
np.mean(hr_5f_lst), np.mean(hrf) 
# (0.3058518518518519, 0.7499999999999999)

In [ ]:
# All bounds
np.mean(hr_5f_lst), np.mean(hrf)
# (0.3149563227792578, 0.45714285714285713)

In [ ]:
import json

import sys
sys.path.append("..")

from utils.event import event_to_pm
from utils.common import load_event
from utils.event import expand_score

In [4]:
from utils.event import decode_token_to_pm, normalize_tp
from utils.decode import decode_ts_tp

In [39]:
with open("../../sonata-dataset-phrase/struct/beethoven-sonata01-1.json") as f:
    struct_event = json.load(f)

with open("../../sonata-dataset-phrase/struct_mark/beethoven-sonata01-1.json") as f:
    struct_bound = json.load(f)

In [38]:
from io import BytesIO

In [35]:
# Phrase Key Analyze
struct_analysis = {}
for sect_name in struct_event:
    struct_analysis[sect_name] = {"bound":[], "key":""}
    
    for i_phrase in range(len(struct_event[sect_name])):
        ts = struct_event[sect_name][i_phrase]['time_signature']
        tp = normalize_tp(struct_event[sect_name][i_phrase]['tempo'])
        
        tokens = [f"ts-{ts}", f"tp-{tp}"]
        for entry in struct_event[sect_name][i_phrase]['event']:
            tokens += entry + ['bar']
        
        pm = decode_token_to_pm(tokens)
    
        with BytesIO() as f:
            pm.write(f)
            f.seek(0)
            midi_string = f.read()
            
        score = music21.midi.translate.midiStringToStream(midi_string)
        key = score.analyze('key')

In [1]:
import music21

In [37]:
for i in range(0, 7):
    score = music21.converter.parse(f'{sect_name}-phrase-{i}.mid')
    key = score.analyze('key')
    print(i, key.tonic.name, key.mode)

0 A- major
1 B- minor
2 C minor
3 A- major
4 F minor
5 F minor
6 F minor


In [25]:
with open("../../sonata-dataset-phrase/struct_mark/beethoven-sonata01-1.json") as f:
    struct_bound = json.load(f)

In [ ]:
struct_event, struct_bound = main(composer, file_base, cfg)
with open(os.path.join(cfg.data_dir, "struct", f"{composer}-{file_base}.json"), "w") as f:
    json.dump(struct_event, f)

with open(os.path.join(cfg.data_dir, "struct_mark", f"{composer}-{file_base}.json"), "w") as f:
    json.dump(struct_bound, f)